<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [594]:
!mkdir '/content/wailua_opaekaa2018'
!mkdir '/content/wailua_opaekaa2018/Images'
!mkdir '/content/wailua_opaekaa2018/Labels'

In [595]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2018.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2018.zip
   creating: /content/CCAP/content/wailua_opaekaa2018/Images/
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/1_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/1_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/1_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/513_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/513_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/513_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/975_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/975_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2018/Images/975_773.tif  
   creating: /content/CCAP/content/wailua_opaekaa2018/Labels/
  inflating: /content/CCAP/content/wailua_opaekaa2018/Labels/1_1.tif  
  inflating: /content/CCAP/content/wailua_o

In [596]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/wailua_opaekaa2018/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/wailua_opaekaa2018/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [597]:
df = pd.read_csv('/content/CCAP/content/wailua_opaekaa2018/wailua_opaekaa2018.csv')

In [598]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [599]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
7,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,13,0.029213
8,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,14,0.001320
9,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,15,0.025204
19,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,13,0.053467
20,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,14,0.011425
21,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,15,0.065632
31,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,13,0.046638
32,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,15,0.003185
33,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,16,0.005936
43,513,1025,1,513,4,464127.603189,465356.414997,2.439898e+06,2.441126e+06,13,0.033253


In [600]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [601]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
7,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,13,0.029213,1_1.tif
8,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,14,0.001320,1_1.tif
9,513,513,1,1,1,464127.560235,465356.383570,2.441126e+06,2.442355e+06,15,0.025204,1_1.tif
19,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,13,0.053467,513_1.tif
20,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,14,0.011425,513_1.tif
21,1025,513,513,1,2,465356.376214,466585.187027,2.441126e+06,2.442355e+06,15,0.065632,513_1.tif
31,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,13,0.046638,975_1.tif
32,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,15,0.003185,975_1.tif
33,1487,513,975,1,3,466465.202544,467694.013619,2.441126e+06,2.442355e+06,16,0.005936,975_1.tif
43,513,1025,1,513,4,464127.603189,465356.414997,2.439898e+06,2.441126e+06,13,0.033253,1_513.tif


In [602]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/wailua_opaekaa2018/Images'
destination_images = '/content/wailua_opaekaa2018/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/wailua_opaekaa2018/Images
Copied 1_1.tif to /content/wailua_opaekaa2018/Images
Copied 1_1.tif to /content/wailua_opaekaa2018/Images
Copied 513_1.tif to /content/wailua_opaekaa2018/Images
Copied 513_1.tif to /content/wailua_opaekaa2018/Images
Copied 513_1.tif to /content/wailua_opaekaa2018/Images
Copied 975_1.tif to /content/wailua_opaekaa2018/Images
Copied 975_1.tif to /content/wailua_opaekaa2018/Images
Copied 975_1.tif to /content/wailua_opaekaa2018/Images
Copied 1_513.tif to /content/wailua_opaekaa2018/Images
Copied 1_513.tif to /content/wailua_opaekaa2018/Images
Copied 1_513.tif to /content/wailua_opaekaa2018/Images
Copied 513_513.tif to /content/wailua_opaekaa2018/Images
Copied 513_513.tif to /content/wailua_opaekaa2018/Images
Copied 513_513.tif to /content/wailua_opaekaa2018/Images
Copied 975_513.tif to /content/wailua_opaekaa2018/Images
Copied 975_513.tif to /content/wailua_opaekaa2018/Images
Copied 975_513.tif to /content/wailua_opaekaa2018/Images
Copi

In [603]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/wailua_opaekaa2018/Labels'
destination_labels = '/content/wailua_opaekaa2018/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/wailua_opaekaa2018/Labels
Copied 1_1.tif to /content/wailua_opaekaa2018/Labels
Copied 1_1.tif to /content/wailua_opaekaa2018/Labels
Copied 513_1.tif to /content/wailua_opaekaa2018/Labels
Copied 513_1.tif to /content/wailua_opaekaa2018/Labels
Copied 513_1.tif to /content/wailua_opaekaa2018/Labels
Copied 975_1.tif to /content/wailua_opaekaa2018/Labels
Copied 975_1.tif to /content/wailua_opaekaa2018/Labels
Copied 975_1.tif to /content/wailua_opaekaa2018/Labels
Copied 1_513.tif to /content/wailua_opaekaa2018/Labels
Copied 1_513.tif to /content/wailua_opaekaa2018/Labels
Copied 1_513.tif to /content/wailua_opaekaa2018/Labels
Copied 513_513.tif to /content/wailua_opaekaa2018/Labels
Copied 513_513.tif to /content/wailua_opaekaa2018/Labels
Copied 513_513.tif to /content/wailua_opaekaa2018/Labels
Copied 975_513.tif to /content/wailua_opaekaa2018/Labels
Copied 975_513.tif to /content/wailua_opaekaa2018/Labels
Copied 975_513.tif to /content/wailua_opaekaa2018/Labels
Copi

In [604]:
# NEW images and labels folder: how many files are there?
new_img = '/content/wailua_opaekaa2018/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/wailua_opaekaa2018/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [605]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/wailua_opaekaa2018_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/wailua_opaekaa2018_subset.csv


In [606]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/wailua_opaekaa2018'
zip_file_path = '/content/wailua_opaekaa2018_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/wailua_opaekaa2018_subset.zip


In [607]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet


# 2019 tiles

In [608]:
!mkdir '/content/wailua_opaekaa2019'
!mkdir '/content/wailua_opaekaa2019/Images'
!mkdir '/content/wailua_opaekaa2019/Labels'

In [609]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2019.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2019.zip
   creating: /content/CCAP/content/wailua_opaekaa2019/Images/
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/1_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/1_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/1_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/513_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/513_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/513_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/975_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/975_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2019/Images/975_773.tif  
   creating: /content/CCAP/content/wailua_opaekaa2019/Labels/
  inflating: /content/CCAP/content/wailua_opaekaa2019/Labels/1_1.tif  
  inflating: /content/CCAP/content/wailua_o

In [610]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/wailua_opaekaa2019/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/wailua_opaekaa2019/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [611]:
df = pd.read_csv('/content/CCAP/content/wailua_opaekaa2019/wailua_opaekaa2019.csv')

In [612]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [613]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
7,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,13,0.029213
8,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,14,0.001320
9,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,15,0.025204
19,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,13,0.053326
20,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,14,0.011425
21,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,15,0.065083
31,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,13,0.046638
32,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,15,0.003185
33,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,16,0.005936
43,513,1025,1,513,4,464127.612670,465356.393057,2.439898e+06,2.441126e+06,13,0.033253


In [614]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [615]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
7,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,13,0.029213,1_1.tif
8,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,14,0.001320,1_1.tif
9,513,513,1,1,1,464127.576710,465356.358548,2.441126e+06,2.442355e+06,15,0.025204,1_1.tif
19,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,13,0.053326,513_1.tif
20,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,14,0.011425,513_1.tif
21,1025,513,513,1,2,465356.418402,466585.209764,2.441126e+06,2.442355e+06,15,0.065083,513_1.tif
31,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,13,0.046638,975_1.tif
32,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,15,0.003185,975_1.tif
33,1487,513,975,1,3,466465.204417,467693.999598,2.441126e+06,2.442355e+06,16,0.005936,975_1.tif
43,513,1025,1,513,4,464127.612670,465356.393057,2.439898e+06,2.441126e+06,13,0.033253,1_513.tif


In [616]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/wailua_opaekaa2019/Images'
destination_images = '/content/wailua_opaekaa2019/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/wailua_opaekaa2019/Images
Copied 1_1.tif to /content/wailua_opaekaa2019/Images
Copied 1_1.tif to /content/wailua_opaekaa2019/Images
Copied 513_1.tif to /content/wailua_opaekaa2019/Images
Copied 513_1.tif to /content/wailua_opaekaa2019/Images
Copied 513_1.tif to /content/wailua_opaekaa2019/Images
Copied 975_1.tif to /content/wailua_opaekaa2019/Images
Copied 975_1.tif to /content/wailua_opaekaa2019/Images
Copied 975_1.tif to /content/wailua_opaekaa2019/Images
Copied 1_513.tif to /content/wailua_opaekaa2019/Images
Copied 1_513.tif to /content/wailua_opaekaa2019/Images
Copied 1_513.tif to /content/wailua_opaekaa2019/Images
Copied 513_513.tif to /content/wailua_opaekaa2019/Images
Copied 513_513.tif to /content/wailua_opaekaa2019/Images
Copied 513_513.tif to /content/wailua_opaekaa2019/Images
Copied 975_513.tif to /content/wailua_opaekaa2019/Images
Copied 975_513.tif to /content/wailua_opaekaa2019/Images
Copied 975_513.tif to /content/wailua_opaekaa2019/Images
Copi

In [617]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/wailua_opaekaa2019/Labels'
destination_labels = '/content/wailua_opaekaa2019/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/wailua_opaekaa2019/Labels
Copied 1_1.tif to /content/wailua_opaekaa2019/Labels
Copied 1_1.tif to /content/wailua_opaekaa2019/Labels
Copied 513_1.tif to /content/wailua_opaekaa2019/Labels
Copied 513_1.tif to /content/wailua_opaekaa2019/Labels
Copied 513_1.tif to /content/wailua_opaekaa2019/Labels
Copied 975_1.tif to /content/wailua_opaekaa2019/Labels
Copied 975_1.tif to /content/wailua_opaekaa2019/Labels
Copied 975_1.tif to /content/wailua_opaekaa2019/Labels
Copied 1_513.tif to /content/wailua_opaekaa2019/Labels
Copied 1_513.tif to /content/wailua_opaekaa2019/Labels
Copied 1_513.tif to /content/wailua_opaekaa2019/Labels
Copied 513_513.tif to /content/wailua_opaekaa2019/Labels
Copied 513_513.tif to /content/wailua_opaekaa2019/Labels
Copied 513_513.tif to /content/wailua_opaekaa2019/Labels
Copied 975_513.tif to /content/wailua_opaekaa2019/Labels
Copied 975_513.tif to /content/wailua_opaekaa2019/Labels
Copied 975_513.tif to /content/wailua_opaekaa2019/Labels
Copi

In [618]:
# NEW images and labels folder: how many files are there?
new_img = '/content/wailua_opaekaa2019/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/wailua_opaekaa2019/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [619]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/wailua_opaekaa2019_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/wailua_opaekaa2019_subset.csv


In [620]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/wailua_opaekaa2019'
zip_file_path = '/content/wailua_opaekaa2019_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/wailua_opaekaa2019_subset.zip


In [621]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet


# 2020 tiles

In [622]:
!mkdir '/content/wailua_opaekaa2020'
!mkdir '/content/wailua_opaekaa2020/Images'
!mkdir '/content/wailua_opaekaa2020/Labels'

In [623]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2020.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/wailua_opaekaa2020.zip
   creating: /content/CCAP/content/wailua_opaekaa2020/Images/
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/1_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/1_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/1_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/513_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/513_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/513_773.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/975_1.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/975_513.tif  
  inflating: /content/CCAP/content/wailua_opaekaa2020/Images/975_773.tif  
   creating: /content/CCAP/content/wailua_opaekaa2020/Labels/
  inflating: /content/CCAP/content/wailua_opaekaa2020/Labels/1_1.tif  
  inflating: /content/CCAP/content/wailua_o

In [624]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/wailua_opaekaa2020/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/wailua_opaekaa2020/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [625]:
df = pd.read_csv('/content/CCAP/content/wailua_opaekaa2020/wailua_opaekaa2020.csv')

In [626]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [627]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
7,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,13,0.029236
8,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,14,0.001236
9,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,15,0.025314
19,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,13,0.053326
20,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,14,0.011425
21,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,15,0.065258
31,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,13,0.046638
32,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,15,0.003185
33,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,16,0.005936
43,513,1025,1,513,4,464127.573037,465356.389161,2.439898e+06,2.441126e+06,13,0.033310


In [628]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [629]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
7,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,13,0.029236,1_1.tif
8,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,14,0.001236,1_1.tif
9,513,513,1,1,1,464127.611128,465356.401113,2.441126e+06,2.442355e+06,15,0.025314,1_1.tif
19,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,13,0.053326,513_1.tif
20,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,14,0.011425,513_1.tif
21,1025,513,513,1,2,465356.377237,466585.144181,2.441126e+06,2.442355e+06,15,0.065258,513_1.tif
31,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,13,0.046638,975_1.tif
32,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,15,0.003185,975_1.tif
33,1487,513,975,1,3,466465.219435,467694.002394,2.441126e+06,2.442355e+06,16,0.005936,975_1.tif
43,513,1025,1,513,4,464127.573037,465356.389161,2.439898e+06,2.441126e+06,13,0.033310,1_513.tif


In [630]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/wailua_opaekaa2020/Images'
destination_images = '/content/wailua_opaekaa2020/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/wailua_opaekaa2020/Images
Copied 1_1.tif to /content/wailua_opaekaa2020/Images
Copied 1_1.tif to /content/wailua_opaekaa2020/Images
Copied 513_1.tif to /content/wailua_opaekaa2020/Images
Copied 513_1.tif to /content/wailua_opaekaa2020/Images
Copied 513_1.tif to /content/wailua_opaekaa2020/Images
Copied 975_1.tif to /content/wailua_opaekaa2020/Images
Copied 975_1.tif to /content/wailua_opaekaa2020/Images
Copied 975_1.tif to /content/wailua_opaekaa2020/Images
Copied 1_513.tif to /content/wailua_opaekaa2020/Images
Copied 1_513.tif to /content/wailua_opaekaa2020/Images
Copied 1_513.tif to /content/wailua_opaekaa2020/Images
Copied 513_513.tif to /content/wailua_opaekaa2020/Images
Copied 513_513.tif to /content/wailua_opaekaa2020/Images
Copied 513_513.tif to /content/wailua_opaekaa2020/Images
Copied 975_513.tif to /content/wailua_opaekaa2020/Images
Copied 975_513.tif to /content/wailua_opaekaa2020/Images
Copied 975_513.tif to /content/wailua_opaekaa2020/Images
Copi

In [631]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/wailua_opaekaa2020/Labels'
destination_labels = '/content/wailua_opaekaa2020/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/wailua_opaekaa2020/Labels
Copied 1_1.tif to /content/wailua_opaekaa2020/Labels
Copied 1_1.tif to /content/wailua_opaekaa2020/Labels
Copied 513_1.tif to /content/wailua_opaekaa2020/Labels
Copied 513_1.tif to /content/wailua_opaekaa2020/Labels
Copied 513_1.tif to /content/wailua_opaekaa2020/Labels
Copied 975_1.tif to /content/wailua_opaekaa2020/Labels
Copied 975_1.tif to /content/wailua_opaekaa2020/Labels
Copied 975_1.tif to /content/wailua_opaekaa2020/Labels
Copied 1_513.tif to /content/wailua_opaekaa2020/Labels
Copied 1_513.tif to /content/wailua_opaekaa2020/Labels
Copied 1_513.tif to /content/wailua_opaekaa2020/Labels
Copied 513_513.tif to /content/wailua_opaekaa2020/Labels
Copied 513_513.tif to /content/wailua_opaekaa2020/Labels
Copied 513_513.tif to /content/wailua_opaekaa2020/Labels
Copied 975_513.tif to /content/wailua_opaekaa2020/Labels
Copied 975_513.tif to /content/wailua_opaekaa2020/Labels
Copied 975_513.tif to /content/wailua_opaekaa2020/Labels
Copi

In [632]:
# NEW images and labels folder: how many files are there?
new_img = '/content/wailua_opaekaa2020/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/wailua_opaekaa2020/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [633]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/wailua_opaekaa2020_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/wailua_opaekaa2020_subset.csv


In [634]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/wailua_opaekaa2020'
zip_file_path = '/content/wailua_opaekaa2020_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/wailua_opaekaa2020_subset.zip


In [635]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet


## Next 2018 tiles

In [678]:
!mkdir '/content/waipio2018'
!mkdir '/content/waipio2018/Images'
!mkdir '/content/waipio2018/Labels'

In [679]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2018.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2018.zip
   creating: /content/CCAP/content/waipio2018/Images/
  inflating: /content/CCAP/content/waipio2018/Images/1_1.tif  
  inflating: /content/CCAP/content/waipio2018/Images/1_1025.tif  
  inflating: /content/CCAP/content/waipio2018/Images/1_1430.tif  
  inflating: /content/CCAP/content/waipio2018/Images/1_513.tif  
  inflating: /content/CCAP/content/waipio2018/Images/513_1.tif  
  inflating: /content/CCAP/content/waipio2018/Images/513_1025.tif  
  inflating: /content/CCAP/content/waipio2018/Images/513_1430.tif  
  inflating: /content/CCAP/content/waipio2018/Images/513_513.tif  
  inflating: /content/CCAP/content/waipio2018/Images/735_1.tif  
  inflating: /content/CCAP/content/waipio2018/Images/735_1025.tif  
  inflating: /content/CCAP/content/waipio2018/Images/735_1430.tif  
  inflating: /content/CCAP/content/waipio2018/Images/735_513.tif  
   creating: /content/CCAP/content/waipio2018/Labels/
  inf

In [680]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/waipio2018/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/waipio2018/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 12
The number of files in the folder is: 12


In [681]:
df = pd.read_csv('/content/CCAP/content/waipio2018/waipio2018.csv')

In [682]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [683]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
10,1025,513,513,1,2,227696.337735,228925.145203,2.226524e+06,2.227753e+06,13,0.001442
11,1025,513,513,1,2,227696.337735,228925.145203,2.226524e+06,2.227753e+06,15,0.180859
20,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,13,0.005611
21,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,14,0.001446
22,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,15,0.155392
31,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,13,0.042999
32,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,14,0.027473
33,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,15,0.005039
41,1025,1025,513,513,5,227696.331640,228925.134335,2.225296e+06,2.226525e+06,13,0.197365
42,1025,1025,513,513,5,227696.331640,228925.134335,2.225296e+06,2.226525e+06,14,0.096992


In [684]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [685]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
10,1025,513,513,1,2,227696.337735,228925.145203,2.226524e+06,2.227753e+06,13,0.001442,513_1.tif
11,1025,513,513,1,2,227696.337735,228925.145203,2.226524e+06,2.227753e+06,15,0.180859,513_1.tif
20,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,13,0.005611,735_1.tif
21,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,14,0.001446,735_1.tif
22,1247,513,735,1,3,228229.138873,229457.982053,2.226524e+06,2.227753e+06,15,0.155392,735_1.tif
31,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,13,0.042999,1_513.tif
32,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,14,0.027473,1_513.tif
33,513,1025,1,513,4,226467.524894,227696.312578,2.225296e+06,2.226525e+06,15,0.005039,1_513.tif
41,1025,1025,513,513,5,227696.331640,228925.134335,2.225296e+06,2.226525e+06,13,0.197365,513_513.tif
42,1025,1025,513,513,5,227696.331640,228925.134335,2.225296e+06,2.226525e+06,14,0.096992,513_513.tif


In [686]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/waipio2018/Images'
destination_images = '/content/waipio2018/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 513_1.tif to /content/waipio2018/Images
Copied 513_1.tif to /content/waipio2018/Images
Copied 735_1.tif to /content/waipio2018/Images
Copied 735_1.tif to /content/waipio2018/Images
Copied 735_1.tif to /content/waipio2018/Images
Copied 1_513.tif to /content/waipio2018/Images
Copied 1_513.tif to /content/waipio2018/Images
Copied 1_513.tif to /content/waipio2018/Images
Copied 513_513.tif to /content/waipio2018/Images
Copied 513_513.tif to /content/waipio2018/Images
Copied 513_513.tif to /content/waipio2018/Images
Copied 735_513.tif to /content/waipio2018/Images
Copied 735_513.tif to /content/waipio2018/Images
Copied 735_513.tif to /content/waipio2018/Images
Copied 1_1025.tif to /content/waipio2018/Images
Copied 1_1025.tif to /content/waipio2018/Images
Copied 1_1025.tif to /content/waipio2018/Images
Copied 513_1025.tif to /content/waipio2018/Images
Copied 513_1025.tif to /content/waipio2018/Images
Copied 513_1025.tif to /content/waipio2018/Images
Copied 1_1430.tif to /content/waipio

In [687]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/waipio2018/Labels'
destination_labels = '/content/waipio2018/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 513_1.tif to /content/waipio2018/Labels
Copied 513_1.tif to /content/waipio2018/Labels
Copied 735_1.tif to /content/waipio2018/Labels
Copied 735_1.tif to /content/waipio2018/Labels
Copied 735_1.tif to /content/waipio2018/Labels
Copied 1_513.tif to /content/waipio2018/Labels
Copied 1_513.tif to /content/waipio2018/Labels
Copied 1_513.tif to /content/waipio2018/Labels
Copied 513_513.tif to /content/waipio2018/Labels
Copied 513_513.tif to /content/waipio2018/Labels
Copied 513_513.tif to /content/waipio2018/Labels
Copied 735_513.tif to /content/waipio2018/Labels
Copied 735_513.tif to /content/waipio2018/Labels
Copied 735_513.tif to /content/waipio2018/Labels
Copied 1_1025.tif to /content/waipio2018/Labels
Copied 1_1025.tif to /content/waipio2018/Labels
Copied 1_1025.tif to /content/waipio2018/Labels
Copied 513_1025.tif to /content/waipio2018/Labels
Copied 513_1025.tif to /content/waipio2018/Labels
Copied 513_1025.tif to /content/waipio2018/Labels
Copied 1_1430.tif to /content/waipio

In [688]:
# NEW images and labels folder: how many files are there?
new_img = '/content/waipio2018/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/waipio2018/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 8
The number of files in the folder is: 8


In [689]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/waipio2018_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/waipio2018_subset.csv


In [690]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/waipio2018'
zip_file_path = '/content/waipio2018_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/waipio2018_subset.zip


In [691]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet


## next 2019 tiles

In [692]:
!mkdir '/content/waipio2019'
!mkdir '/content/waipio2019/Images'
!mkdir '/content/waipio2019/Labels'

In [693]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2019.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2019.zip
   creating: /content/CCAP/content/waipio2019/Images/
  inflating: /content/CCAP/content/waipio2019/Images/1_1.tif  
  inflating: /content/CCAP/content/waipio2019/Images/1_1025.tif  
  inflating: /content/CCAP/content/waipio2019/Images/1_1430.tif  
  inflating: /content/CCAP/content/waipio2019/Images/1_513.tif  
  inflating: /content/CCAP/content/waipio2019/Images/513_1.tif  
  inflating: /content/CCAP/content/waipio2019/Images/513_1025.tif  
  inflating: /content/CCAP/content/waipio2019/Images/513_1430.tif  
  inflating: /content/CCAP/content/waipio2019/Images/513_513.tif  
  inflating: /content/CCAP/content/waipio2019/Images/735_1.tif  
  inflating: /content/CCAP/content/waipio2019/Images/735_1025.tif  
  inflating: /content/CCAP/content/waipio2019/Images/735_1430.tif  
  inflating: /content/CCAP/content/waipio2019/Images/735_513.tif  
   creating: /content/CCAP/content/waipio2019/Labels/
  inf

In [694]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/waipio2019/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/waipio2019/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 12
The number of files in the folder is: 12


In [695]:
df = pd.read_csv('/content/CCAP/content/waipio2019/waipio2019.csv')

In [696]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [697]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
10,1025,513,513,1,2,227696.344251,228925.145604,2.226524e+06,2.227753e+06,13,0.001442
11,1025,513,513,1,2,227696.344251,228925.145604,2.226524e+06,2.227753e+06,15,0.180859
20,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,13,0.005611
21,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,14,0.001446
22,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,15,0.155983
31,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,13,0.042999
32,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,14,0.027473
33,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,15,0.005039
41,1025,1025,513,513,5,227696.357866,228925.136537,2.225296e+06,2.226525e+06,13,0.196861
42,1025,1025,513,513,5,227696.357866,228925.136537,2.225296e+06,2.226525e+06,14,0.097019


In [698]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [699]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
10,1025,513,513,1,2,227696.344251,228925.145604,2.226524e+06,2.227753e+06,13,0.001442,513_1.tif
11,1025,513,513,1,2,227696.344251,228925.145604,2.226524e+06,2.227753e+06,15,0.180859,513_1.tif
20,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,13,0.005611,735_1.tif
21,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,14,0.001446,735_1.tif
22,1247,513,735,1,3,228229.119048,229457.914208,2.226524e+06,2.227753e+06,15,0.155983,735_1.tif
31,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,13,0.042999,1_513.tif
32,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,14,0.027473,1_513.tif
33,513,1025,1,513,4,226467.499187,227696.292330,2.225296e+06,2.226525e+06,15,0.005039,1_513.tif
41,1025,1025,513,513,5,227696.357866,228925.136537,2.225296e+06,2.226525e+06,13,0.196861,513_513.tif
42,1025,1025,513,513,5,227696.357866,228925.136537,2.225296e+06,2.226525e+06,14,0.097019,513_513.tif


In [700]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/waipio2019/Images'
destination_images = '/content/waipio2019/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 513_1.tif to /content/waipio2019/Images
Copied 513_1.tif to /content/waipio2019/Images
Copied 735_1.tif to /content/waipio2019/Images
Copied 735_1.tif to /content/waipio2019/Images
Copied 735_1.tif to /content/waipio2019/Images
Copied 1_513.tif to /content/waipio2019/Images
Copied 1_513.tif to /content/waipio2019/Images
Copied 1_513.tif to /content/waipio2019/Images
Copied 513_513.tif to /content/waipio2019/Images
Copied 513_513.tif to /content/waipio2019/Images
Copied 513_513.tif to /content/waipio2019/Images
Copied 735_513.tif to /content/waipio2019/Images
Copied 735_513.tif to /content/waipio2019/Images
Copied 735_513.tif to /content/waipio2019/Images
Copied 1_1025.tif to /content/waipio2019/Images
Copied 1_1025.tif to /content/waipio2019/Images
Copied 1_1025.tif to /content/waipio2019/Images
Copied 513_1025.tif to /content/waipio2019/Images
Copied 513_1025.tif to /content/waipio2019/Images
Copied 513_1025.tif to /content/waipio2019/Images
Copied 1_1430.tif to /content/waipio

In [701]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/waipio2019/Labels'
destination_labels = '/content/waipio2019/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 513_1.tif to /content/waipio2019/Labels
Copied 513_1.tif to /content/waipio2019/Labels
Copied 735_1.tif to /content/waipio2019/Labels
Copied 735_1.tif to /content/waipio2019/Labels
Copied 735_1.tif to /content/waipio2019/Labels
Copied 1_513.tif to /content/waipio2019/Labels
Copied 1_513.tif to /content/waipio2019/Labels
Copied 1_513.tif to /content/waipio2019/Labels
Copied 513_513.tif to /content/waipio2019/Labels
Copied 513_513.tif to /content/waipio2019/Labels
Copied 513_513.tif to /content/waipio2019/Labels
Copied 735_513.tif to /content/waipio2019/Labels
Copied 735_513.tif to /content/waipio2019/Labels
Copied 735_513.tif to /content/waipio2019/Labels
Copied 1_1025.tif to /content/waipio2019/Labels
Copied 1_1025.tif to /content/waipio2019/Labels
Copied 1_1025.tif to /content/waipio2019/Labels
Copied 513_1025.tif to /content/waipio2019/Labels
Copied 513_1025.tif to /content/waipio2019/Labels
Copied 513_1025.tif to /content/waipio2019/Labels
Copied 1_1430.tif to /content/waipio

In [702]:
# NEW images and labels folder: how many files are there?
new_img = '/content/waipio2019/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/waipio2019/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 8
The number of files in the folder is: 8


In [703]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/waipio2019_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/waipio2019_subset.csv


In [704]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/waipio2019'
zip_file_path = '/content/waipio2019_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/waipio2019_subset.zip


In [705]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet


## next 2020 tiles

In [706]:
!mkdir '/content/waipio2020'
!mkdir '/content/waipio2020/Images'
!mkdir '/content/waipio2020/Labels'

In [707]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2020.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/waipio2020.zip
   creating: /content/CCAP/content/waipio2020/Images/
  inflating: /content/CCAP/content/waipio2020/Images/1_1.tif  
  inflating: /content/CCAP/content/waipio2020/Images/1_1025.tif  
  inflating: /content/CCAP/content/waipio2020/Images/1_1430.tif  
  inflating: /content/CCAP/content/waipio2020/Images/1_513.tif  
  inflating: /content/CCAP/content/waipio2020/Images/513_1.tif  
  inflating: /content/CCAP/content/waipio2020/Images/513_1025.tif  
  inflating: /content/CCAP/content/waipio2020/Images/513_1430.tif  
  inflating: /content/CCAP/content/waipio2020/Images/513_513.tif  
  inflating: /content/CCAP/content/waipio2020/Images/735_1.tif  
  inflating: /content/CCAP/content/waipio2020/Images/735_1025.tif  
  inflating: /content/CCAP/content/waipio2020/Images/735_1430.tif  
  inflating: /content/CCAP/content/waipio2020/Images/735_513.tif  
   creating: /content/CCAP/content/waipio2020/Labels/
  inf

In [708]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/waipio2020/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/waipio2020/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 12
The number of files in the folder is: 12


In [709]:
df = pd.read_csv('/content/CCAP/content/waipio2020/waipio2020.csv')

In [710]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [711]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
10,1025,513,513,1,2,227696.359333,228925.158854,2.226525e+06,2.227753e+06,13,0.001392
11,1025,513,513,1,2,227696.359333,228925.158854,2.226525e+06,2.227753e+06,15,0.179516
20,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,13,0.005611
21,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,14,0.001446
22,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,15,0.155983
31,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,13,0.043251
32,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,14,0.027527
33,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,15,0.005039
41,1025,1025,513,513,5,227696.339585,228925.140003,2.225296e+06,2.226524e+06,13,0.197048
42,1025,1025,513,513,5,227696.339585,228925.140003,2.225296e+06,2.226524e+06,14,0.097019


In [712]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [713]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
10,1025,513,513,1,2,227696.359333,228925.158854,2.226525e+06,2.227753e+06,13,0.001392,513_1.tif
11,1025,513,513,1,2,227696.359333,228925.158854,2.226525e+06,2.227753e+06,15,0.179516,513_1.tif
20,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,13,0.005611,735_1.tif
21,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,14,0.001446,735_1.tif
22,1247,513,735,1,3,228229.108583,229457.930734,2.226525e+06,2.227753e+06,15,0.155983,735_1.tif
31,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,13,0.043251,1_513.tif
32,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,14,0.027527,1_513.tif
33,513,1025,1,513,4,226467.507195,227696.325102,2.225296e+06,2.226524e+06,15,0.005039,1_513.tif
41,1025,1025,513,513,5,227696.339585,228925.140003,2.225296e+06,2.226524e+06,13,0.197048,513_513.tif
42,1025,1025,513,513,5,227696.339585,228925.140003,2.225296e+06,2.226524e+06,14,0.097019,513_513.tif


In [714]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/waipio2020/Images'
destination_images = '/content/waipio2020/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 513_1.tif to /content/waipio2020/Images
Copied 513_1.tif to /content/waipio2020/Images
Copied 735_1.tif to /content/waipio2020/Images
Copied 735_1.tif to /content/waipio2020/Images
Copied 735_1.tif to /content/waipio2020/Images
Copied 1_513.tif to /content/waipio2020/Images
Copied 1_513.tif to /content/waipio2020/Images
Copied 1_513.tif to /content/waipio2020/Images
Copied 513_513.tif to /content/waipio2020/Images
Copied 513_513.tif to /content/waipio2020/Images
Copied 513_513.tif to /content/waipio2020/Images
Copied 735_513.tif to /content/waipio2020/Images
Copied 735_513.tif to /content/waipio2020/Images
Copied 735_513.tif to /content/waipio2020/Images
Copied 1_1025.tif to /content/waipio2020/Images
Copied 1_1025.tif to /content/waipio2020/Images
Copied 1_1025.tif to /content/waipio2020/Images
Copied 513_1025.tif to /content/waipio2020/Images
Copied 513_1025.tif to /content/waipio2020/Images
Copied 513_1025.tif to /content/waipio2020/Images
Copied 1_1430.tif to /content/waipio

In [715]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/waipio2020/Labels'
destination_labels = '/content/waipio2020/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 513_1.tif to /content/waipio2020/Labels
Copied 513_1.tif to /content/waipio2020/Labels
Copied 735_1.tif to /content/waipio2020/Labels
Copied 735_1.tif to /content/waipio2020/Labels
Copied 735_1.tif to /content/waipio2020/Labels
Copied 1_513.tif to /content/waipio2020/Labels
Copied 1_513.tif to /content/waipio2020/Labels
Copied 1_513.tif to /content/waipio2020/Labels
Copied 513_513.tif to /content/waipio2020/Labels
Copied 513_513.tif to /content/waipio2020/Labels
Copied 513_513.tif to /content/waipio2020/Labels
Copied 735_513.tif to /content/waipio2020/Labels
Copied 735_513.tif to /content/waipio2020/Labels
Copied 735_513.tif to /content/waipio2020/Labels
Copied 1_1025.tif to /content/waipio2020/Labels
Copied 1_1025.tif to /content/waipio2020/Labels
Copied 1_1025.tif to /content/waipio2020/Labels
Copied 513_1025.tif to /content/waipio2020/Labels
Copied 513_1025.tif to /content/waipio2020/Labels
Copied 513_1025.tif to /content/waipio2020/Labels
Copied 1_1430.tif to /content/waipio

In [716]:
# NEW images and labels folder: how many files are there?
new_img = '/content/waipio2020/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/waipio2020/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 8
The number of files in the folder is: 8


In [717]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/waipio2020_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/waipio2020_subset.csv


In [718]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/waipio2020'
zip_file_path = '/content/waipio2020_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/waipio2020_subset.zip


In [719]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/planet'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/planet
